In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## Generate Simulated Image Set for Photogrammetry

Using the default starting objects of Blender, this Jupyter module generates an **image set to be used with photogrammetry software**.

You can export this notebook as Python script (**File**&rarr;**Download as**&rarr;**Python (.py)**) and execute it from Blender (**Chose Screen layout**&rarr;**Open**&rarr;**Run Script**).

See the example at the end of this notebook.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import math
import os

In [ ]:
def set_up_sc(sc, **kwargs):
    """Set up some parameters of a scene, sc ('bpy.context.scene')."""
    wpx = kwargs.pop('wpx', 320)
    hpx = kwargs.pop('hpx', 240)
    antialias = kwargs.pop('antialias', '8')
    bw = kwargs.pop('bw', True)
    extension = kwargs.pop('extension', 'TIFF')
    sc.render.resolution_percentage = 100
    sc.render.resolution_x = wpx
    sc.render.resolution_y = hpx
    sc.render.antialiasing_samples = antialias
    sc.render.use_overwrite = True
    sc.render.image_settings.file_format = extension
    if bw:
        sc.render.image_settings.color_mode = 'BW'
    else:
        sc.render.image_settings.color_mode = 'RGB'

In [ ]:
def num_str_zeros(num, n_digs, matlab=False):
    """Return a string that contains a sequence n-zeros followed by
    num ('int') as 'str', for example, num_str_zeros(89, 4) returns
    '0089'. From fwdimaging.ipynb Jupyter notebook."""
    if matlab:  # Begin the numeration with 1 in the filename.
        num += 1
    str_num = ''
    for i in range(n_digs - len(str(num))):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
def shoot(cam, sc, filename, outpath='ph_blend_set/'):
    """Render a scene ('bpy.context.scene') with a camera
    ('bpy.types.Camera') and save the result in outpath
    ('str') route as PNG file."""
    sc.camera = cam
    sc.render.filepath = outpath + filename
    bpy.ops.render.render(write_still=True)

In [ ]:
def take_photos(cam, sc, l_loc_rot, **kwargs):
    """Take photos with the cam ('bpy.types.Camera') in the
    coordinates and the angles referenced as tuples of l_loc_rot
    ('list')."""
    prefix = kwargs.pop('prefix', 'view_')
    len_llocrot = len(l_loc_rot)
    for i in range(len_llocrot):
        loc, rot = l_loc_rot[i]
        cam.location = loc
        for j in range(3):  # 3 by each rotation.
            cam.rotation_euler[j] = math.radians(rot[j])
        shoot(cam, sc, prefix + num_str_zeros(i, len(str(len_llocrot)),
                                              matlab=True))

In [ ]:
def range_float(a, b, step):
    """Emulate the np.arange() of the numpy Python module.
    
    With this function, IT IS NOT NECESSARY to install numpy for
    Blender."""
    l = list()
    if a > b:
        while a > b:
            l.append(a)
            a -= abs(step)
    else:
        if a < b:
            while a < b:
                l.append(a)
                a += abs(step)
    return l

In [ ]:
def walk_sphere(**kwargs):
    """Return a list of tuples with the (x, y, z) coordinates and
    rotation angles of a camera that moves along a circular path in
    different heights and radius of a sphere.
    
    Example
    >>> l = walk_sphere()
    >>> l
    [((7.984359711335656, 0.0, 0.5000000000000012),
      (86.41667830152802, 0, 90)),
       ...,
     ((3.3777275704249043, -5.850395766102116, 4.2853034711634965),
      (57.61111886768535, 0, 390.0))]
    >>> (x, y, z), (rotx, roty, rotz) = l[0]
    >>> rotz
    90
    """
    r = kwargs.pop('r', 8)  # The radius in 'NONE' units.
    # Number of photos during movement in xy-plane.
    hsteps = kwargs.pop('hsteps', 6)
    # Number of xy-planes of movement.
    vsteps = kwargs.pop('vsteps', 3)
    # First height of the camera in 'NONE' units.
    hinit = kwargs.pop('hinit', 0.5)
    # Maximum height of the camera. It not should be greater that
    # the radius, r ('float').
    hend = kwargs.pop('hend', 6)
    # In Blender, a simple way to find out the phase change angle is
    # placing the camera ('bpy.types.Camera') in the first (x, y, z)
    # coordinate and manually changing the Y rotation angle of
    # 'XYZ' rotation order.
    dphase = kwargs.pop('dphase', 90)
    # Phi is the angle measured from the z-Axis to the xy-plane.
    phi_init = 90 - math.degrees(math.asin(hinit / float(r)))
    phi_end = 90 - math.degrees(math.asin(hend / float(r)))
    phi_step = (phi_init ) / float(vsteps)
    range_phi = range_float(phi_init, phi_end, phi_step)
    theta_step = 360 / float(hsteps)
    range_theta = range_float(0, 360, theta_step)
    l_coord = list()
    for phi in range_phi:
        for theta in range_theta:
            # Parameterization of a sphere [1].
            #
            # [1] Carmen, M. (2007). Parametrizaciones. Retrieved from
            # http://www.dm.uba.ar/materias/complementos_analisis_Mae/2007/2/parametrizaciones.pdf
            x = r * math.cos(math.radians(theta)) * math.sin(math.radians(phi))
            y = r * math.sin(math.radians(theta)) * math.sin(math.radians(phi))
            z = r * math.cos(math.radians(phi))
            rotx = phi
            roty = 0
            rotz = theta + dphase
            l_coord.append(((x, y, z), (rotx, roty, rotz)))
    return l_coord

### Example

In [ ]:
# UNCOMMENT ALL BELOW to render the scene.
# scene = bpy.context.scene
# set_up_sc(scene, extension='PNG', wpx=640, hpx=480, bw=False)
# cam = bpy.data.objects[0]
# take_photos(cam, scene, walk_sphere())